In [1]:
import pandas as pd
import numpy as np  

In [2]:
# Load the data
df = pd.read_csv('.csv/cleaned_data.csv', index_col = 0)
model_data = df.copy()
pd.set_option('display.max_column', None)

In [3]:
model_data.replace(np.inf, np.nan, inplace=True)

In [4]:
# Convert categorical columns to string first, then to category
model_data['NewExist_Encoded'] = model_data['NewExist'].map({1.0: 1, 2.0: 0}, na_action='ignore') \
                                                      .fillna('missing') \
                                                      .astype('str') \
                                                      .astype('category')

model_data['UrbanRural_Encoded'] = model_data['UrbanRural'].map({1.0: 1, 2.0: 0}, na_action='ignore') \
                                                          .fillna('missing') \
                                                          .astype('str') \
                                                          .astype('category')

model_data['MIS_Status_Encoded'] = model_data['MIS_Status'].map({'CHGOFF': 0, 'PIF': 1}) \
                                                          .fillna('missing') \
                                                          .astype('str') \
                                                          .astype('category')

model_data['RevLineCr_Encoded'] = model_data['RevLineCr'].map({'N': 0, 'Y': 1}, na_action='ignore') \
                                                      .fillna('missing') \
                                                      .astype('str') \
                                                      .astype('category')

model_data['LowDoc_Encoded'] = model_data['LowDoc'].map({'N': 0, 'Y': 1}) \
                                                  .fillna('missing') \
                                                  .astype('str') \
                                                  .astype('category')

model_data['FranchiseCode_Encoded'] = model_data['FranchiseCode_Encoded'].map({'No': 0, 'Yes': 1}) \
                                                                    .fillna('missing') \
                                                                    .astype('str') \
                                                                    .astype('category')

model_data['RealEstate_Backed'] = model_data['RealEstate_Backed'].map({'No': 0, 'Yes': 1}) \
                                                              .fillna('missing') \
                                                              .astype('str') \
                                                              .astype('category')

model_data['CreateJob_Encoded'] = model_data['CreateJob'].apply(lambda x: 1 if x > 0 else 0) \
                                                      .astype('str') \
                                                      .astype('category')

model_data['RetainedJob_Encoded'] = model_data['RetainedJob'].apply(lambda x: 1 if x > 0 else 0) \
                                                        .astype('str') \
                                                        .astype('category')

model_data['State'] = model_data['State'].fillna('missing').astype('str').astype('category')

model_data['StateRisk'] = model_data['StateRisk'].fillna('missing').astype('str').astype('category')

model_data['Region'] = model_data['Region'].fillna('missing').astype('str').astype('category')

model_data['NAICS_class_code'] = model_data['NAICS_class_code'].fillna('missing').astype('str').astype('category')


In [5]:
model_data.loc[:, 'EmployeeLoanRatio'] = model_data.apply(
    lambda row: round(row['GrAppv']) if pd.isna(row['EmployeeLoanRatio']) else round(row['EmployeeLoanRatio']),
    axis=1
).astype('int64')

In [6]:
model_data.drop(labels=['LoanNr_ChkDgt', 'Name', 'City', 'Bank', 'BankState', 'TermDays', 'ApprovalDate', 'ApprovalFY', 'Zip', 'DisbursementDate', 
                      'DisbursementGross','NewExist', 'RetainedJob', 'LowDoc' ,'UrbanRural', 'RevLineCr', 'ChgOffDate',
                      'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'SBA_Appv', 'Industry', 'Recession',
                       'ApprovalDateYear', 'ChgOffDateYear', 'ApprovalDateMonth', 'DisbursementDateYear',
                       'LoanDateEnd'], axis=1, inplace=True )

In [7]:
model_data.head()

,State,NAICS,Term,NoEmp,CreateJob,FranchiseCode,GrAppv,NAICS_class_code,FranchiseCode_Encoded,RealEstate_Backed,Region,EmployeeLoanRatio,StateRisk,NewExist_Encoded,UrbanRural_Encoded,MIS_Status_Encoded,RevLineCr_Encoded,LowDoc_Encoded,CreateJob_Encoded,RetainedJob_Encoded
0,IN,451120,84,4,0,1,60000,45,0,0,Eastern,15000.0,Medium,0.0,missing,1,0.0,1,0,0
1,IN,722410,60,2,0,1,40000,72,0,0,Eastern,20000.0,Medium,0.0,missing,1,0.0,1,0,0
2,IN,621210,180,7,0,1,287000,62,0,0,Eastern,41000.0,Medium,1.0,missing,1,0.0,0,0,0
3,OK,0,60,2,0,1,35000,81,0,0,Eastern,17500.0,Low,1.0,missing,1,0.0,1,0,0
4,FL,0,240,14,7,1,229000,81,0,1,Eastern,16357.0,High,1.0,missing,1,0.0,0,1,1


In [8]:
from sklearn.model_selection import train_test_split

# Split the data into features and target
X = model_data.drop(columns=['MIS_Status_Encoded'])
y = model_data['MIS_Status_Encoded']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [10]:
import catboost as cb

# Define model
cb_model = cb.CatBoostClassifier(#task_type="GPU",
                                random_state=42,
                                scale_pos_weight= 1.0,
                                random_strength= 10.0,
                                learning_rate= 0.38,
                                l2_leaf_reg= 4.5,
                                iterations= 1000,
                                depth= 11,
                                border_count= 128,
                                bagging_temperature= 0.0,
                                verbose=100,  # Log every 100 iterations
                            )

categorical_features_indices = [X_train.columns.get_loc(col) for col in X_train.select_dtypes(include=['category']).columns]

cb_model.fit(X_train, y_train, cat_features=categorical_features_indices)

cb_train_score = cb_model.score(X_train, y_train)

print("Training Score:", round(cb_train_score, 2))

# Store model parameters in a DataFrame
best_cb_params = cb_model.get_params()  # Get model's hyperparameters
best_cb_params_df = pd.DataFrame([best_cb_params])

# Save to CSV
best_cb_params_df.to_csv("best_cb_params.csv", index=False)

print("Best parameters saved to CSV successfully!")

0:	learn: 0.4170822	total: 1.54s	remaining: 25m 41s
100:	learn: 0.1392130	total: 1m 23s	remaining: 12m 26s
200:	learn: 0.1207031	total: 2m 45s	remaining: 10m 57s
300:	learn: 0.1079831	total: 4m 19s	remaining: 10m 2s
400:	learn: 0.0987944	total: 5m 37s	remaining: 8m 23s
500:	learn: 0.0903173	total: 7m 5s	remaining: 7m 3s
600:	learn: 0.0834116	total: 8m 38s	remaining: 5m 44s
700:	learn: 0.0769166	total: 10m 8s	remaining: 4m 19s
800:	learn: 0.0718537	total: 11m 39s	remaining: 2m 53s
900:	learn: 0.0669095	total: 13m 1s	remaining: 1m 25s
999:	learn: 0.0623995	total: 14m 18s	remaining: 0us
Training Score: 0.98
Best parameters saved to CSV successfully!


In [11]:
from sklearn.metrics import classification_report

# Make predictions
cb_y_pred = cb_model.predict(X_test)

# Calculate the accuracy
cb_classification_report = classification_report(y_test, cb_y_pred)

print(cb_classification_report)

              precision    recall  f1-score   support

           0       0.84      0.81      0.83     31564
           1       0.96      0.97      0.96    147922

    accuracy                           0.94    179486
   macro avg       0.90      0.89      0.90    179486
weighted avg       0.94      0.94      0.94    179486



In [20]:
from sklearn.model_selection import cross_val_score

# Evaluate model using cross-validation
lgb_roc_scores = cross_val_score(cb_model, X_train, y_train, cv=5, scoring='roc_auc', error_score='raise')
print(f'ROC-AUC: {lgb_roc_scores.mean()}')

CatBoostError: features data: pandas.DataFrame column 'State' has dtype 'category' but is not in  cat_features list

In [12]:
from catboost import CatBoostClassifier, Pool

cat_boost_params = {
    'scale_pos_weight': 1.0,
    'random_strength': 10.0,
    'learning_rate': 0.24,
    'l2_leaf_reg': 5.0,
    'iterations': 900,
    'depth': 12,
    'border_count': 128,
    'bagging_temperature': 0.3
}

cat_boost_best = CatBoostClassifier(
    random_state=42,
    verbose=100,  # Log every 100 iterations
    **cat_boost_params  # Apply your optimal parameters
)

# Detect categorical features
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

# Get the indices of these columns
categorical_features_indices = [X.columns.get_loc(col) for col in categorical_features]

# Create data pools
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
test_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

cat_boost_best.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)

0:	learn: 0.4887134	test: 0.4884369	best: 0.4884369 (0)	total: 1.65s	remaining: 24m 39s
100:	learn: 0.1484334	test: 0.1548779	best: 0.1548779 (100)	total: 1m 10s	remaining: 9m 19s
200:	learn: 0.1267665	test: 0.1487949	best: 0.1487927 (197)	total: 3m 10s	remaining: 11m 1s
300:	learn: 0.1120995	test: 0.1485540	best: 0.1484838 (263)	total: 5m 26s	remaining: 10m 50s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1484838015
bestIteration = 263

Shrink model to first 264 iterations.


In [13]:
from sklearn.metrics import classification_report

# Make predictions
cat_y_pred = cat_boost_best.predict(X_test)

# Calculate the accuracy
cat_classification_report = classification_report(y_test, cat_y_pred)

print(cat_classification_report)

              precision    recall  f1-score   support

           0       0.85      0.82      0.84     31564
           1       0.96      0.97      0.97    147922

    accuracy                           0.94    179486
   macro avg       0.91      0.89      0.90    179486
weighted avg       0.94      0.94      0.94    179486



In [ ]:
from catboost import cv

cv_params = {
    'random_strength': 10.0,
    'learning_rate': 0.24,
    'l2_leaf_reg': 5.0,
    'iterations': 900,
    'depth': 12,
    'border_count': 128,
    'bagging_temperature': 0.3,
    'verbose': 100,
    'random_state': 42,
    'loss_function': 'Logloss'  # Use Logloss for binary classification
}


cv_data = cv(
    params=cv_params,
    pool=train_pool,
    fold_count=5,  # 5-Fold Cross-Validation
    early_stopping_rounds=50,
    stratified=True,  # Ensure stratified folds
    partition_random_seed=42
)

# Evaluate AUC
best_auc = cv_data['test-AUC-mean'].max()  # Access AUC after cross-validation
print(f"Best Mean ROC-AUC Score: {best_auc}")




Training on fold [0/5]
0:	learn: 0.5038384	test: 0.5040188	best: 0.5040188 (0)	total: 1.16s	remaining: 17m 20s
100:	learn: 0.1482943	test: 0.1532802	best: 0.1532802 (100)	total: 1m 13s	remaining: 9m 42s
200:	learn: 0.1218902	test: 0.1470689	best: 0.1470380 (197)	total: 2m 28s	remaining: 8m 36s

bestTest = 0.1468807731
bestIteration = 233

Training on fold [1/5]
0:	learn: 0.5039818	test: 0.5040046	best: 0.5040046 (0)	total: 458ms	remaining: 6m 51s
100:	learn: 0.1471899	test: 0.1557606	best: 0.1557606 (100)	total: 56.3s	remaining: 7m 25s
200:	learn: 0.1221346	test: 0.1495586	best: 0.1495460 (198)	total: 2m 8s	remaining: 7m 26s

bestTest = 0.1493960027
bestIteration = 214

Training on fold [2/5]
0:	learn: 0.5041550	test: 0.5036262	best: 0.5036262 (0)	total: 459ms	remaining: 6m 53s
100:	learn: 0.1462146	test: 0.1554358	best: 0.1554358 (100)	total: 55.8s	remaining: 7m 21s
200:	learn: 0.1199934	test: 0.1503672	best: 0.1503648 (197)	total: 2m 8s	remaining: 7m 28s

bestTest = 0.1501865921
best

KeyError: 'test-AUC-mean'

In [17]:
# Print available columns
print(cv_data.columns)

Index(['iterations', 'test-Logloss-mean', 'test-Logloss-std',
       'train-Logloss-mean', 'train-Logloss-std'],
      dtype='object')


In [22]:
# Print available columns to check keys
print(cv_data.columns)

# Access the correct metric name
best_logloss = cv_data['test-Logloss-mean'].min()
print(f"Best Mean Logloss: {best_logloss}")


Index(['iterations', 'test-Logloss-mean', 'test-Logloss-std',
       'train-Logloss-mean', 'train-Logloss-std'],
      dtype='object')
Best Mean Logloss: 0.14599035796803722
